In [1]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM
import string
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from peft import LoraConfig
import torch

In [2]:
## モデル名

model_name = "cyberagent/calm2-7b-chat"

In [3]:
## トークナイザーのインポート

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast = True
)

In [4]:
## モデルのインポート
## リソースが足りないので、1/4量子化

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = 'auto', # 複数のGPUがあるときに、それらを均等に使用する
    load_in_8bit = True, # 精度と引き換えに、パラメータを8bitに省略したLLMを使用して計算を軽量化
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
request_key = "USER:"
response_key = "ASSISTANT:"
chat_template = string.Template(
    "USER:${user}\nASSISTANT:${assistant}"
)

In [6]:
sample_text = chat_template.safe_substitute({
    "user": "人気タレント・タモリの本名は何でしょう？",
     "assistant": "" # AIに回答させるために空けておく
})

In [7]:
def generate(model, text):
    input_ids = tokenizer.encode(
        text,
        return_tensors = 'pt',
        add_special_tokens = True
    ).to(model.device)
    output_ids = model.generate(
        input_ids,
        max_new_tokens = 100,
        # do_sample = True,
        # temperature = 0.8,
    )
    print(
        tokenizer.decode(
            output_ids[0],
            skip_special_tokens = True
        )
    )

In [8]:
%%time
generate(model, sample_text)

USER:人気タレント・タモリの本名は何でしょう？
ASSISTANT: タモリの本名は「森田一義」です。
CPU times: user 2.23 s, sys: 313 ms, total: 2.55 s
Wall time: 2.97 s


In [9]:
## 外部のデータセットをインポート

rs = "izumi-lab/llm-japanese-dataset"
datasets = load_dataset(
    rs,
    split = 'train'
)
datasets = datasets.train_test_split(test_size = 0.1)
train_datasets = datasets['train']
eval_datasets = datasets['test']

In [10]:
## 学習データセットの整形プロセス

def update_datasets(example):
    output_texts = []
    for instruction_, input_, output_ in zip(example['instruction'], example['input'], example['output']):
        text = chat_template.safe_substitute({
            "user": "{0}\n{1}".format(instruction_, input_),
            "assistant": "なんだいっ！！こんなこともわからないのかい！！仕方ないねっ！！！\n{0}".format(output_)
        })
        output_texts.append(text)
    return output_texts

In [11]:
## ??

collator = DataCollatorForCompletionOnlyLM(
    instruction_template = tokenizer.encode(request_key, add_special_tokens = False),
    response_template = tokenizer.encode(response_key, add_special_tokens = False),
    tokenizer = tokenizer
)

In [12]:
peft_config = LoraConfig(
    peft_type = 'LORA',
    task_type = 'CAUSAL_LM',
    r = 8,
    target_modules = 'all-linear',
    lora_alpha = 8,
    lora_dropout = 0.0,
)

In [13]:
## アダプタの付与
## 量子化したモデルをそのままファインチューニングできないため
## 全体チューニングではなく、出力換算するための行列を修飾する(Lora)
## 実際にtrainされるのは行列の部分だけ

model.add_adapter(peft_config)

In [14]:
%%time
generate(model, sample_text)

USER:人気タレント・タモリの本名は何でしょう？
ASSISTANT: タモリの本名は「森田一義」です。
CPU times: user 2.3 s, sys: 33 ms, total: 2.34 s
Wall time: 2.61 s


In [15]:
%%time

## 訓練オブジェクトの定義

trainer = SFTTrainer(
    model,
    train_dataset = train_datasets.select(range(1000)),
    eval_dataset = eval_datasets.select(range(100)),
    args = SFTConfig(
        output_dir = "./tmp",
        evaluation_strategy = "epoch",
        per_device_train_batch_size = 4,
        per_device_eval_batch_size = 4,
        learning_rate = 1e-5,
        num_train_epochs = 3.0,
        lr_scheduler_type = "linear",
        warmup_ratio = 0.0,
        logging_strategy = "epoch",
        save_strategy = "epoch",
        report_to = "all"
    ),
    formatting_func = update_datasets,
    data_collator = collator,
    max_seq_length = 1024,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

CPU times: user 446 ms, sys: 44.1 ms, total: 490 ms
Wall time: 368 ms


In [16]:
%%time

## 訓練実行

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan
3,0.000000,nan


/usr/local/lib/python3.11/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


CPU times: user 5min 48s, sys: 1min 1s, total: 6min 50s
Wall time: 6min 54s


TrainOutput(global_step=750, training_loss=0.0, metrics={'train_runtime': 414.081, 'train_samples_per_second': 7.245, 'train_steps_per_second': 1.811, 'total_flos': 1.2154012348416e+16, 'train_loss': 0.0, 'epoch': 3.0})

In [17]:
%%time
generate(model, sample_text)

USER:人気タレント・タモリの本名は何でしょう？
ASSISTANT:
CPU times: user 130 ms, sys: 2.19 ms, total: 132 ms
Wall time: 131 ms
